In [8]:
"""
Created on Tue Nov 15 2022
@author: Juliane Rosemeier
""";

<big> <big> <big>   1D single scale example 


This notebook solves
\begin{equation}
\dot x = -x , \qquad x(0) = 1.
\end{equation}
The exact solution is given by
\begin{equation}
x(t) = \exp(-t) .
\end{equation}
The differential equation is used to test the Multi-level Parareal method without averaging. A V-cycle is implemented.

In [9]:
"""Import modules""" 

import numpy as np
import matplotlib.pyplot as plt


In [10]:
"""Define the RHS of the problem""" 

def rhs(x):
    return -x


In [11]:
"""Implement Runge-Kutta method of second order (RK2)""" 

def RK2(f,x,delta_t): 
    """One step with the RK2 method"""
    
    # compute stages
    k1 = f(x)
    k2 = f(x + 0.5*delta_t*k1) 
    
    # new time step
    y = x + delta_t*k2 
    
    return y 


def RK2_on_interval(f,x0,grid):
    """Apply the RK2 method on an interval
        f - RHS
        x0 - initial value
        grid - time grid"""
    
    # number of grid points
    num_points = len(grid)
    
    # array for the RK solution
    RK_sol = np.zeros(num_points)
    RK_sol[0] = x0
    
    # compute RK solutions on the grid points
    for n in range(num_points-1):
        tau = grid[n+1] - grid[n]
        RK_sol[n+1] = RK2(f,RK_sol[n],tau)
        
    return RK_sol 

In [12]:
"""Discrete l1-norm of the error""" 
def error(v1,v2):
    vec_E = abs(v1-v2)/len(v1)
    E = 0.
    for n in range(len(v1)):
        E += vec_E[n]
    return E

In [13]:
"""Implementation of recursive Parareal V-cycle""" 

def parareal_Vcycle(x0, f, dt, len_interval, level, L_max, coarsening_factor):
    """This function provides a recursive implementation of the Parareal V-cycle. 
        x0 - initial value, 
        f - RHS of the prblem, 
        dt - coarse time step, 
        len_interval - length of the solution interval, 
        level - current level,
        L_max - coarsest level, 
        coarsening_factor - coarsening factor of the method"""
    
    
    # coarse time grid
    time_grid = np.arange(0, len_interval + 0.1*dt, dt)
    len_grid = len(time_grid)
    
    # fine time grid
    fine_dt = dt/coarsening_factor
    fine_grid = np.arange(0, dt+0.1*fine_dt, fine_dt)
    
    # array for fine solution
    fine_solution = np.zeros(len_grid)
    fine_solution[0] = x0
    
    # array for Parareal iterate
    iterate = np.zeros(len_grid)
    iterate[0] = x0
    
    
    # compute initial guess
    initial_guess = RK2_on_interval(rhs, x0, time_grid)
    
    # compute fine solution, work can be done in parallel

    for steps in range(len_grid-1):
        if level == 1:
            f_sol = RK2_on_interval(rhs, initial_guess[steps], fine_grid)
            fine_solution[steps+1] = f_sol[len(f_sol)-1]
        else: 
            fine_solution[steps+1] = parareal_Vcycle(initial_guess[steps], rhs, fine_dt, dt, level-1, \
                                                     L_max, coarsening_factor)        
           
    # compute iterate

    for steps in range(len_grid-1):
        new_RK_step = RK2_on_interval(rhs, iterate[steps], np.array([0., dt]))
        old_RK_step = RK2_on_interval(rhs, initial_guess[steps], np.array([0., dt]))
        iterate[steps+1] = new_RK_step[len(new_RK_step)-1] + fine_solution[steps+1] \
                            - old_RK_step[len(new_RK_step)-1]
        
       
    # compute the error and plot the results for the coarsest level    
    if level == L_max:
        
        # compute error
        print('number of levels: ', L_max+1, '  error: ',  error(iterate, np.exp(-time_grid)))
        
        # plot results
        #plt.plot(time_grid, np.exp(-time_grid))
        #plt.plot(time_grid, initial_guess,'o')
        #plt.plot(time_grid, iterate,'.')
        #plt.show()
    
    return iterate[len(iterate)-1]



In [ ]:
"""Test the Multi-level Parareal V-cycle. The coarsest level has the same step size in all cases."""

# coarsening factor in the tests
c_factor = 10

# coarse time step
dt = .25

# initial value
x0 = 1.

# tests with a different the number of levels

L_max1 = 1
parareal_Vcycle(x0, rhs, dt, 2., L_max1, L_max1, c_factor);

L_max2 = 2
parareal_Vcycle(x0, rhs, dt, 2., L_max2, L_max2, c_factor);

L_max3 = 3
parareal_Vcycle(x0, rhs, dt, 2., L_max3, L_max3, c_factor);

L_max4 = 4
parareal_Vcycle(x0, rhs, dt, 2., L_max4, L_max4, c_factor);

L_max5 = 5
parareal_Vcycle(x0, rhs, dt, 2., L_max5, L_max5, c_factor);

L_max6 = 6
parareal_Vcycle(x0, rhs, dt, 2., L_max6, L_max6, c_factor);

L_max7 = 7
parareal_Vcycle(x0, rhs, dt, 2., L_max7, L_max7, c_factor);


number of levels:  2   error:  1.2566212807763046e-05
number of levels:  3   error:  1.9562958164422008e-05
number of levels:  4   error:  1.9807099440426344e-05
number of levels:  5   error:  1.9809587023590493e-05
number of levels:  6   error:  1.9809615854133382e-05
number of levels:  7   error:  1.9809616125891306e-05
